In [101]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [109]:
# Reading in File . . .
file_path = "train.json"
json_data = []

df = pd.read_json(file_path, lines=True)
print(df.head())

       0.0      36.6      36.7      38.0      38.2  38.300000000000004  \
0   3.0074   81.2963   83.7169  119.6888  115.8881            413.4768   
1  13.6523  361.5547  119.7225  170.9554   84.0531            273.8909   
2   7.1155   80.2869  169.7490  149.4092  132.8215            198.8544   
3   2.0998  235.5117  161.2402  241.3578  243.2237             89.7297   
4  16.2152  384.2739  174.5714   84.3117  137.9726            261.8213   

       39.0      39.1      39.2       39.6  ...    466.0    523.0    523.1  \
0  136.8640  817.5328   86.4743   477.6116  ...  15.8853   1.5231   4.9432   
1  175.9083  665.0791  588.2624    84.1962  ...   6.9459  26.9344   9.0805   
2  150.5273  112.4005  335.4482   675.7621  ...   9.3406  18.8055   7.0021   
3  640.7394  121.9373  396.3036  1138.3905  ...  26.2306   8.1640  26.6106   
4  365.8102   96.3555  219.9515   211.2868  ...  39.9084   4.7010  50.1098   

     523.4     524.4    524.5     554.2     554.4    1046.0       target  
0   9.2590 

In [110]:
# Combining targets that are tonally equivalent . . .   
    
def combine_equal(df):
    tonal_equ = {
                "A#": "Bb",
                "B#": "C",
                "C#": "Db",
                "D#": "Eb",
                "E#": "F",
                "F#": "Gb",
                "G#": "Ab"
            }
    
    df[['note', 'modality']] = pd.DataFrame(df.target.tolist(), index=df.index)
    df["note"] = df['note'].replace(tonal_equ)
    df["target"] = df[["note", "modality"]].astype(str).apply(' '.join, axis=1)
    df.drop(["note", "modality"], axis=1, inplace=True)

combine_equal(df)

In [111]:
# Normalizing using z-score standardization method . . .

target_column = df['target']
features = df.select_dtypes(include=[np.number])
normalized_features = (features - features.mean()) / features.std()
df_normalized = pd.concat([normalized_features, target_column], axis=1)
print(df_normalized.head()) 

        0.0      36.6      36.7      38.0      38.2  38.300000000000004  \
0 -0.028187 -0.099215 -0.037191 -0.118219 -0.108142           -0.120322   
1 -0.028186 -0.099127 -0.037188 -0.118203 -0.108152           -0.120366   
2 -0.028187 -0.099215 -0.037183 -0.118209 -0.108137           -0.120389   
3 -0.028188 -0.099167 -0.037184 -0.118181 -0.108105           -0.120424   
4 -0.028186 -0.099120 -0.037183 -0.118230 -0.108136           -0.120370   

       39.0      39.1      39.2      39.6  ...     466.0     523.0     523.1  \
0 -0.125820 -0.115750 -0.114163 -0.118979  ... -0.139936 -0.177052 -0.184882   
1 -0.125809 -0.115793 -0.114020 -0.119089  ... -0.139938 -0.177044 -0.184881   
2 -0.125816 -0.115949 -0.114092 -0.118923  ... -0.139938 -0.177047 -0.184881   
3 -0.125672 -0.115947 -0.114075 -0.118794  ... -0.139933 -0.177050 -0.184875   
4 -0.125753 -0.115954 -0.114125 -0.119053  ... -0.139929 -0.177051 -0.184868   

      523.4     524.4     524.5     554.2     554.4    1046.0    tar

In [112]:
# function to randomly drop features to make data into perfect square for CNN input . . .

def reduce_to_perfect_square(df):
    
    num_features = df.shape[1]
    max_square = int(np.sqrt(num_features)) ** 2
    
    if max_square == num_features:
        return df
    else:
        features_to_drop = num_features - max_square-1
        dropped_features = np.random.choice(df.columns, size=features_to_drop, replace=False)
        reduced_df = df.drop(columns=dropped_features)
        
        return reduced_df

reduced_df = reduce_to_perfect_square(df_normalized)
print(reduced_df["target"])


0        E major
1        E major
2        E major
3        E major
4       Db major
          ...   
9842     C major
9843    Ab major
9844    Gb minor
9845    Ab major
9846     D minor
Name: target, Length: 9847, dtype: object


In [113]:
#zero padding? to achieve a 32 by 32 image . . . (previously only 31 features, with more features use more)

unique_classes = reduced_df['target'].nunique()
print(unique_classes)
# padding = np.zeros((reduced_df.shape[0], 1))
# df_padded = pd.concat([reduced_df, pd.DataFrame(padding, columns=['Pad'+str(i) for i in range(1)])], axis=1)

24


In [114]:
# reshaping the data into the image size . . . will need to change for larger sets
X = reduced_df.drop('target', axis=1).values
X_reshaped = X.reshape(-1, 31, 31, 1) 

In [115]:
# Splitting into train test validation sets
label_encoder = LabelEncoder()
reduced_df['target_encoded'] = label_encoder.fit_transform(reduced_df['target'])
y = reduced_df['target_encoded'].values
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

In [116]:
# Defining the CNN architecture . . .
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(12, 12), strides=(2, 2), activation='relu', input_shape=(31, 31, 1)),
    layers.Conv2D(64, kernel_size=(5, 5), strides=(1, 2), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(len(np.unique(df['target'])), activation='softmax')
])

lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.05,
    decay_steps=2000,
    decay_rate=0.95,
    staircase=True)

optimizer = optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/Users/maxsivry/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 10, 10, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 6, 3, 64)       │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 3, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 1024)           │       197,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 24)             │        24,600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 278,136 (1.06 MB)

 Trainable params: 278,136 (1.06 MB)

 Non-trainable params: 0 (0.00 B)

In [117]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)

# Evaluate the model
performance = model.evaluate(X_val, y_val)
print(f'Validation Loss: {performance[0]}, Validation Accuracy: {performance[1]}')

Epoch 1/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.0702 - loss: 21.7603 - val_accuracy: 0.0584 - val_loss: 3.1332
Epoch 2/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.0728 - loss: 3.0982 - val_accuracy: 0.0716 - val_loss: 3.1185
Epoch 3/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.0665 - loss: 3.1084 - val_accuracy: 0.0584 - val_loss: 3.1186
Epoch 4/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.0664 - loss: 3.1058 - val_accuracy: 0.0741 - val_loss: 3.1286
Epoch 5/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0785 - loss: 3.1091 - val_accuracy: 0.0594 - val_loss: 3.1263
Epoch 6/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.0742 - loss: 3.1037 - val_accuracy: 0.0741 - val_loss: 3.1193
Epoch 7/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.0704 - loss: 3.1015 - val_accuracy: 0.0741 - val_loss: 3.1239
Epoch 8/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.0724 - loss: 3.1025 - val_acc

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0749 - loss: 3.1162
Validation Loss: 3.118629217147827, Validation Accuracy: 0.07411167770624161
